# Bayesian Parameter Estimation of a Fluorescence-to-Protein Calibration Factor 

In [120]:
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import emcee
import pandas as pd
import theano.tensor as tt
import statsmodels.tools.numdiff as smnd
import mscl.plotting
import mscl.mcmc
import multiprocessing 
import bokeh.io
import corner
import scipy.special
import scipy.optimize
import scipy.stats
import bokeh.plotting
# import bebi103.pm
bokeh.io.output_notebook()
%matplotlib inline
num_cpus = multiprocessing.cpu_count()

Loading BokehJS ...

## Counting By Measuring Fluctuations 

Imagine that we have a cell that has a fixed number of fluorescent proteins.  As this cell divides, these proteins will be Binomially partitioned into the two daughter cells. By examining the difference in fluorescence between the two daughters, we can determine just how "bright" a single molecule should be. 

We can begin by positing that the fluorescence is conserved (production and degradation rates are 0). Mathematically, we can say that 

$$
I_\text{tot} = I_1 + I_2 \tag{1}
$$

where $I_\text{tot}$ is the total fluorescence of the mother cell and $I_1$ and $I_2$ are the intensities of the two daugher cells. As we know this fluorescence comes from a mix of the background fluorescence and the fluorescence of the proteins themselves, we can write that the total intensitiy of the mother cell should be proportional to the total number of proteins $N_\text{tot}$,

$$
I_\text{tot} = \alpha N_\text{tot}, \tag{2}
$$

assuming (for now) that the background fluorescence and the error in our measurement is zero. Assuming that there is a single fluroescence calibration factor and that it has the same value from cell to cell, we can write Eq. 2 for the daughter cells as

$$
I_1 = \alpha N_1\,\, ; \,\, I_2 = \alpha N_2  \tag{3, 4}
$$

which follows the assumption that the protein copy number is conserved as well,

$$
N_\text{tot} = N_1 + N_2 \tag{5}.
$$

When a cell divides, the distribution of the proteins into the two daughter cells is binomial with a partitioning probability $p$,

$$
P(n \, \vert \, N_\text{tot}) = {N_\text{tot}! \over n! (N_\text{tot} - n)!} p^{n}(1 - p)^{N_\text{tot} - n}, \tag{6}
$$

Where we've generalized $N_1$ or $N_2$ as $n$. To examine how the proteins were partitioned amongst the daughter cells, we can look at how different the intensities are between them and relate it to the total intensity of the mother as 

$$
\langle(I_1 - I_2)^2\rangle = \langle (2I_1 - I_\text{tot})^2 \tag{7}
$$

which we can translate to protein copy numbers as 

$$
\langle(I_1 - I_2)^2\rangle = \langle\left(2\alpha N_1 - \alpha N_\text{tot}\right)^2\rangle. \tag{8}
$$

As we are discussing averages in this context, we can examine this more generally as

$$
\langle(I_1 - I_2)^2 \rangle = \alpha(2\langle n \rangle - N_\text{tot})^2 \tag{9}
$$

To finish our calculation, we will have to know the mean and variance of the protein copy number upon division. 





While these two moments of the Binomial distribution are well know, it's useful to derive them explicitly. The mean can be calculated as

$$
\langle n \rangle = \sum\limits_{n = 0}^{N_\text{tot}}n{{N_\text{tot}}\choose{n}}p^{n}(1-p)^{N_\text{tot} - n}. \tag{10}
$$

This can be simplified by realizing that

$$
n {N_\text{tot}! \over n!(N_\text{tot} - n)!} = {N_\text{tot}! \over (n-1)!(N_\text{tot} - n)!} = {N_\text{tot}(N_\text{tot} - 1)! \over (n - 1)!(N_\text{tot} - n)!} \tag{11}
$$

We can now rewrite Eq. 10 as

$$
\langle n \rangle = N_\text{tot} p \sum\limits_{n= 1}^{N_\text{tot}}{{N_\text{tot} - 1}\choose{n - 1}}p^{n - 1}(1 - p)^{N_\text{tot} - n} \tag{12}
$$

By defining $\nu = N_\text{tot} - 1$ and $k = n - 1$, we yield. 

$$
\langle n \rangle = N_\text{tot} p \overbrace{\sum\limits_{k= 0}^{\nu}{{\nu}\choose{k}}p^k(1 - p)^{\nu - k}}^\text{1} = N_\text{tot} p \tag{13}
$$

which is what we would naïvely expect. With this in hand, we can naïvely solve for the variance. We can solve for $\langle n^2 \rangle$ as we did above by realizing 

$$
\langle n^2 \rangle = N_\text{tot}p\sum\limits_{n=1}^{N_\text{tot}}n{{N_\text{tot} - 1}\choose{n -1}}p^{n-1}(1 - p)^{N_\text{tot} - n} = N_\text{tot}p\sum\limits_{k=0}^\nu(k + 1){{\nu}\choose{k}}p^k(1-p)^{\nu - k}  \tag{14},
$$

where we pull the same trick of reparameterizing Eq. 13 for $\nu = N_\text{tot} - 1$ and $k = n - 1$. Simplifying Eq. 14 generates 

$$
\langle n^2 \rangle = (N_\text{tot}p)^2 + N_\text{tot}p(1 - p) \tag{15}
$$

allowing us to express the variance $\sigma^2$ as 

$$
\sigma^2 = \langle n^2 \rangle - \langle n \rangle^2 = N_\text{tot}p(1 - p). \tag{16}
$$



We can make our calculation a bit less verbose aby assuming that partitioning of the proteins is always fair such that $p = 0.5$. This means our mean and variance can be more simply written as 

$$
\langle n \rangle = {N_\text{tot} \over 2}\, ; \, \langle n^2 \rangle= {N_\text{tot}+ N_\text{tot}^2 \over 4}. \tag{17, 18}
$$

We can now return to Eq. 9 and include the mean copy number,

$$
\begin{align}
\langle (I_1 - I_2)^2 \rangle &= 4\alpha^2(\langle n^2 \rangle - \langle n \rangle N_\text{tot}) + (\alpha N_\text{tot})^2\\
& = 4\alpha^2\left({N_\text{tot} + N_\text{tot}^2 \over 4} - {2N_\text{tot}^2 \over 4}\right) + \alpha^2N_\text{tot}^2 \tag{19}.
\end{align}
$$

Some simplification brings us to our result, 

$$
\langle (I_1 - I_2)^2 \rangle = \alpha^2 N_\text{tot} = \alpha I_\text{tot}. \tag{20}
$$
 
This result tells us that the squared difference in the intensity between any two daughter cells should be linearly related to the intensity of the mother cell with a slope of the calibration factor $\alpha$. While this is a very beautiful result, our measurement is a bit more difficult.

## Incorporating Measurement Error

While we were rewarded for our mathematical efforts with Eq. 20, the practical use of this theoretical prediction is a bit more complicated. Primarily, we must be able to incoporate our error in the measurement of the single cell intensities. This means that our Eq. 20 becomes. 

$$
I_\text{tot} = \alpha_0 + \alpha N_\text{tot} + \epsilon \tag{21}
$$

where $\alpha_0$ is the background fluorescence of our sample and $\epsilon$ is our measurement error. In [Rosenfeld et al. 2005]() and [Brewster et al. 2014](), the calibration factor was determined by binning the observed data and then fitting the prediction to the mean, assuming the measurement error was 0. While this is an acceptable approach, choosing the appropriate bin width is difficult and can bias the results. A more robust approach would be to write a heirarchical model which models each cell division event individually and estimate a hyperparameter for the calibration factor. 



### Taking a Bayesian approach

We are interested in estimating two calibration parameters - $\alpha_0$ and $\alpha$ - and (as a consequence), the most likely protein copy numbers $N_1$ and $N_\text{tot}$. As is usually the case, we can begin by writing down Bayes' theorem for the problem, 

$$
P(\alpha_0, \alpha, \sigma, N_1, N_\text{tot}\,\vert\, I_1, I_2) = {P(I_1, I_2\, \vert \, \alpha_0, \alpha, \sigma, N_1, N_\text{tot})P(\alpha_0, \alpha, \sigma, N_1, N_\text{tot}) \over P(I_1, I_2) }. \tag{22}
$$

in which we've introduced the measurement error as $\sigma$ as it's related to the normal distribution of the measurements. Our likelihood can be broken up into three parts $--$ one for $I_1$, for $I_2$, and one for $[N_1, N_\text{tot}]$. As we've discussed already, the partitioning of the proteins into the daughter cells is binomial, allowing us to write

$$
P(N_1\, \vert\, N_\text{tot}, p) = {{N_\text{tot}}\choose{N_1}}p^{N_1}(1 - p)^{N_\text{tot} - N_1}. \tag{23}
$$

By assuming that partitioning is *always* fair (which may be a big assumption), we can simplify Eq. 23 to be

$$
P(N_1\, \vert \, N_\text{tot}, p) = {{N_\text{tot}}\choose{N_1}}2^{-N_\text{tot}}. \tag{24}
$$

It is likely that the measured intensity of a given cell is normally distributed. We can then say that for $I_1$, the  likelihood is gaussian with a variance $\sigma$,

$$
P(I_1 \,\vert\, \alpha_0, \alpha, N_1) = {1 \over \sqrt{2\pi \sigma^2}}\exp\left[{-(I_1 - (\alpha_0 + \alpha N_1)^2 \over 2\sigma^2}\right]. \tag{25}
$$

Similarly, we can write the likelihood for $I_2$ as 

$$
P(I_2 \,\vert\, \alpha_0, \alpha, N_1, N_\text{tot}) = {1 \over \sqrt{2 \pi \sigma^2}}\exp\left[{-(I_2 - (\alpha_0 + \alpha (N_\text{tot} - N_1))^2 \over 2 \sigma^2}\right] \tag{26}
$$

where we've again used the fact that protein copy number is conserved $N_\text{tot} = N_1 + N_2$. 

The priors can be a little more tricky to figure out. To begin simply, we can assume a uniform uninformative prior for the protein copy numbers $N_1$ and $N_\text{tot}$ using our knowledge that they must be discrete,

$$
P(N_1, N_\text{tot}) = \begin{cases} {1 \over n_\text{max} - n_\text{min}} && n \in \mathbb{Z}\, ;\, n_\text{min} \leq n \leq n_\text{max} \,;\, N_1 \leq N_\text{tot}\\
0 && \text{otherwise}
\end{cases}. \tag{27}
$$

Similarly, we can choose uniform priors for our two calibration parameters $\alpha_0$ and $\alpha$ as 

$$
P(\alpha_0, \alpha) = \begin{cases} {1 \over \alpha_\text{max} - \alpha_\text{min}} && \alpha_\text{min} \leq \alpha \leq \alpha_\text{max}\\
0 && \text{otherwise} 
\end{cases}. \tag{28}
$$

As is typically the case, we can assign an improper (i.e. unnormalized) Jeffrey's prior for $\sigma$,

$$
P(\sigma) = {1 \over \sigma}. \tag{29}
$$

With likelihoods and priors, we are ready to write down the complete posterior. For ease, we can ignore the evidence $P(I_1, I_2)$. As we are looking over $k$ cell divisions, we have a set of $[N_1, N_\text{tot}]$ and $[I_1, I_2]$,  

$$
P(\alpha_0, \alpha, \sigma, [N_1, N_\text{tot}]\,\vert\,I_1, I_2) \propto P(\alpha, \alpha_0, [N_1, N_\text{tot}]){1 \over \sigma}\prod\limits_{i}^{k}{{N_{\text{tot},i}}\choose{N_{1,i}}}2^{-N_{\text{tot},i}}\times
\left({1 \over 2\pi \sigma^2}\right)^k\exp\left[{1 \over 2\sigma^2}\sum\limits_{i}^{k}\left((I_{1,i} - (\alpha_0 + \alpha N_{1,i}))^2 + (I_{2, i} - (\alpha_0 + \alpha(N_{\text{tot},i} - N_{1,i})))^2\right)\right], \tag{30}
$$

in which the priors for $\alpha_0, \alpha,$ and $[N_1, N_\text{tot}]$ were left out for notational clarity. 

While we could continue on to sample this distribution using MCMC, we have to be very careful of the priors. For example, with a $2k + 3$ parameter sampling, any degeree of information included in our prior will be greatly amplified and can steer the sampling off into other low-probabability local minima. We can reduce this issue by marginalizing Eq. 30 over $\sigma$,

$$
P(\alpha_0, \alpha, [N_1, N_\text{tot}] \, \vert\, [I_1, I_2]) = \int\limits_0^\infty d\sigma P(\alpha_0, \alpha, \sigma, [N_1, N_\text{tot}]\,\vert\,[I_1, I_2]). \tag{31}
$$

While this is a gnarly integral, the result is well studied and is the Student-t distribution, 

$$
\begin{align}
P(\alpha_0, \alpha, [N_1, N_\text{tot}] \, \vert \, [I_1, I_2]) \propto &\prod\limits_i^k {{N_{\text{tot},i}}\choose{N_{1,i}}}2^{-N_{\text{tot},i}}\times\\
&\left(\sum\limits_i^k \left((I_{1,i} - \alpha_0 - \alpha N_{1,i}))^2 + (I_{2,i} - \alpha_0 - \alpha(N_{\text{tot},i} - N_{1,i}))^2\right)\right)^{-k / 2}. \tag{32}
\end{align}
$$

Rather than jumping immediately into analyzing *real* data, we can test it on something we know.

### A Deterministic approach 

### Analyzing simulated data

To ensure that this type of inference is appropriate, we will generate a fake data set in which the calibration factor and the background fluorescence is known *a priori*. We can choose values that are physically realistic, as is done in the code cell below.

In [308]:
# Set the parameters to generate the data set.
n_cells = 1000 
max_proteins = 100
alpha_opt = 500 

# Set the parent protein copy numbers and partition into children.
n_tot = (np.random.rand(n_cells) * max_proteins).astype(int)
n_1 = np.random.binomial(n_tot, p=0.5)
n_2 = n_tot - n_1

# Compute the intensities.
I_1 = alpha_seed * n_1 + np.random.normal(loc=0, scale=100, size=n_cells) 
I_2 = alpha_seed * n_2 + np.random.normal(loc=0, scale=100, size=n_cells)
I_tot = alpha_seed * n_tot + np.random.normal(loc=0, scale=100, size=n_cells)

# Make it a DataFrame and compute the quantity of interest.
mothers = np.arange(0, n_cells, 1)
df = pd.DataFrame(np.array([n_tot, n_1, n_2, I_1, I_2, I_tot, mothers]).T, columns=['n_tot', 'n_1', 'n_2',
                                                         'I_1', 'I_2', 'I_tot', 'cell_no'])
df.loc[:, 'sq_diff'] = (df['I_1'] - df['I_2'])**2

# Generate the prediction from the theory.
I_tot_range = np.logspace(2, 6, 300)
prediction = alpha_opt * I_tot_range

To prove that this synthetic data is in line with our derivation above, we can plot the raw points along with Eq. 20. 

In [309]:
# Instantiate the figure and add axis labels. 
p = mscl.plotting.boilerplate(plot_width=600, plot_height=400, x_axis_type='log',
                      y_axis_type='log', x_axis_label='I\u2081 + I\u2082',
                      y_axis_label='(I\u2081 - I\u2082)\u00B2')

# Plot the data and the theoretical prediction.
p.circle(x=df['I_tot'], y=df['sq_diff'],color='slategray', 
         alpha=0.4, legend='simulated data')
p.line(x=I_tot_range, y=prediction, color='tomato',
      legend='α\u2080 + α(Ι\u2081 + I\u2082)', line_width=2)

# Add a legend and show the plot
p.legend.location='top_left'

bokeh.io.show(p)

As expected, our theoretical result matches the simulated data. To sample Eq. 32, we will use the package [PyMC3]() which has a wide variety of samplers and simple tools for constructing complicated models.

### A Deterministic Approach

In [310]:
def log_post(alpha, I1, I2, p=0.5):
    # Determine the length.
    k = len(I1)
    
    # Set the deterministic parameters. 
    n1 = I1 / alpha
    n2 = I2 / alpha
    ntot = n1 + n2
    # Compute the binomial coefficient using gamma functions.
    binom = scipy.special.gammaln(ntot + 1).sum() - scipy.special.gammaln(n1 + 1).sum()\
            -scipy.special.gammaln(n2 + 1).sum()
    # Compute the probability
    prob = n1.sum() * np.log(p) + (ntot - n1).sum() * np.log(1 - p)
    
    # Define the prior
    prior = -k * np.log(alpha)
    
    # Compute the log posterior
    return prior + prob + binom
            
def neg_log_post(alpha, I1, I2, p=0.5):
    return -log_post(alpha, I1, I2, p)

In [317]:
# Minimize.
x0 = [0, 2**16 - 1]
popt = scipy.optimize.minimize_scalar(neg_log_post, args=(df['I_1'].values, df['I_2'].values))
                             
alpha_opt = popt.x

In [318]:
# plot the posterior.
alpha_range = np.linspace(1, 1000, 1000)

args = (df['I_1'].values, df['I_2'].values)
post = [log_posterior_deterministic(a, *args) for a in alpha_range]
norm_post = np.exp(post - scipy.special.logsumexp(post))

p = mscl.plotting.boilerplate(x_axis_label='intensity per molecule (a.u.)',
                             y_axis_label='g(α | [Ι\u2081, I\u2082])')
p.line(alpha_range, norm_post, color='dodgerblue', line_width=2)
mscl.plotting.fill_between(p, alpha_range, np.zeros_like(alpha_range), norm_post,
                          color='dodgerblue', alpha=0.2)
p.line((alpha_opt, alpha_opt+0.1), (0, 0.018), color='tomato', line_width=2)
bokeh.io.show(p)

In [319]:
hess = smnd.approx_hess([popt.x], log_post, args=(df['I_1'], df['I_2']))
cov = -np.linalg.inv(hess)
sd = np.sqrt(cov)[0]

In [320]:
# plot the posterior.
alpha_range = np.linspace(1, 1000, 1000)

# compute the gaussian approximation.
gaussian = scipy.stats.norm.pdf(alpha_range, loc=alpha_opt, scale=sd)
gaussian = gaussian / np.sum(gaussian)
args = (df['I_1'].values, df['I_2'].values)
post = [log_posterior_deterministic(a, *args) for a in alpha_range]
norm_post = np.exp(post - scipy.special.logsumexp(post))

p = mscl.plotting.boilerplate(x_axis_label='intensity per molecule (a.u.)',
                             y_axis_label='g(α | [Ι\u2081, I\u2082])')
p.line(alpha_range, norm_post, color='dodgerblue', line_width=2)
p.line(alpha_range, gaussian, color='tomato', line_dash='dotted', line_width=2)
mscl.plotting.fill_between(p, alpha_range, np.zeros_like(alpha_range), norm_post,
                          color='dodgerblue', alpha=0.2)
p.line((alpha_opt, alpha_opt+0.1), (0, 0.018), color='tomato', line_width=2)
bokeh.io.show(p)

In [324]:
# Plot the data, true value, and estimated plot. 
I_tot_range = np.logspace(2, 5, 500)
true_val = alpha_seed * I_tot_range
estimated_val = alpha_opt  * I_tot_range
upper_error = (alpha_opt + sd) * I_tot_range
lower_error = (alpha_opt - sd) * I_tot_range
p = mscl.plotting.boilerplate(x_axis_label='I\u2081 + I\u2082',
                             y_axis_label='(I\u2081 - I\u2082)\u00B2',
                             x_axis_type='log', y_axis_type='log')
p.scatter(df['I_tot'], df['sq_diff'], color='slategray', alpha=0.5, size=3,
         legend='simulated data')

p.line(I_tot_range, estimated_val, color='dodgerblue', legend='best-fit val')
mscl.plotting.fill_between(p, I_tot_range, lower_error, upper_error, color='dodgerblue',
                          alpha=0.5)
p.line(I_tot_range, true_val, color='tomato', legend='true value')
p.legend.location = 'bottom_right'
bokeh.io.show(p)

In [301]:
s

array([[ 23.80083302]])

In [176]:
smnd.approx_hess?

###  A Heirarchical model

In [47]:
with pm.Model() as model:
    # Define the hyperpriors 
    alpha = pm.Uniform('alpha', lower=0, upper=2**16)
    sigma = pm.HalfNormal('sigma')
    ntot = pm.DiscreteUniform('ntot', lower=0, upper=1E6)
    n1 = pm.Binomial('n1', n=ntot, p=0.5, shape=n_cells) 
    n2 = ntot - n1
   
    #Define the observables.
    obs = df.loc[:, ['I_1', 'I_2']].T.values.astype(float)
    
    # Compute the expected value.
    mu1 = alpha * n1
    mu2 = alpha * (ntot  - n1)
    
    # Define the likelihood and sample.
    like_I1 = pm.Normal('like_I1', mu=mu1, observed=df['I_1'].values)
    like_I2 = pm.Normal('like_I2', mu=mu2, observed=df['I_2'].values)
    trace = pm.sample(draws=10000, tune=5000, njobs=4, init='advi')    

Assigned NUTS to alpha_interval__
INFO:pymc3:Assigned NUTS to alpha_interval__
Assigned NUTS to sigma_log__
INFO:pymc3:Assigned NUTS to sigma_log__
Assigned Metropolis to ntot
INFO:pymc3:Assigned Metropolis to ntot
Assigned Metropolis to n1
INFO:pymc3:Assigned Metropolis to n1
  1%|          | 102/15000 [00:07<17:14, 14.40it/s]


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/gchure/anaconda/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Users/gchure/anaconda/lib/py